Import the Dependencies

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [4]:
kaggle_dictionary = json.load(open('kaggle.json'))
kaggle_dictionary

{'username': 'gulshan136', 'key': '2dbfd21fd9a18b4e0325095c16ae9ebd'}

In [6]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [7]:
#set kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 66% 17.0M/25.7M [00:00<00:00, 168MB/s]
100% 25.7M/25.7M [00:00<00:00, 182MB/s]


In [9]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [10]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zipObj:
    zipObj.extractall()

In [11]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


Loading the Dataset

In [13]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [14]:
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [17]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [20]:
data.replace({"sentiment": {"positive":1, "negative":0}}, inplace = True)

In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [24]:
#split data into training and testing data

train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

print("Number of training samples:", len(train_data))
print("Number of testing samples:", len(test_data))

Number of training samples: 40000
Number of testing samples: 10000


In [25]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [26]:
#Tokenize the data

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [27]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [28]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [29]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [31]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [32]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short Term Memory

In [38]:
#Build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [39]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Training the Model

In [42]:
history = model.fit(x_train, y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 324s 640ms/step - accuracy: 0.7079 - loss: 0.5539 - val_accuracy: 0.8267 - val_loss: 0.3916
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 311s 620ms/step - accuracy: 0.8493 - loss: 0.3593 - val_accuracy: 0.8267 - val_loss: 0.3950
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 287s 550ms/step - accuracy: 0.8624 - loss: 0.3337 - val_accuracy: 0.8449 - val_loss: 0.3641
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 342s 590ms/step - accuracy: 0.8736 - loss: 0.3028 - val_accuracy: 0.7980 - val_loss: 0.4308
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 314s 573ms/step - accuracy: 0.8842 - loss: 0.2852 - val_accuracy: 0.8764 - val_loss: 0.3128


In [45]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 44s 138ms/step - accuracy: 0.8810 - loss: 0.3050
Test Loss: 0.30375778675079346
Test Accuracy: 0.8815000057220459


Building Preictive System

In [53]:
def predict_sentiment(review):
  #tokenize the review text
  sequence = tokenizer.texts_to_sequences(review)
  padded_sequence = pad_sequences(sequence, maxlen = 200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

Examples

In [54]:
review1 = "The movie was fantastic"
result = predict_sentiment(review1)
print("The sentiment of this review is", {result})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
The sentiment of this review is {'negative'}


In [56]:
review2 = "The movie was not much good"
result = predict_sentiment(review2)
print("The sentiment of this review is", {result})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
The sentiment of this review is {'negative'}


In [57]:
review3 = "very bad movie"
result = predict_sentiment(review3)
print("The sentiment of this review is", {result})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
The sentiment of this review is {'negative'}
